In [82]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [83]:
df = pd.read_csv('test.csv')
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [84]:
df['title'] = df.Name.str.extract('([a-zA-z]+\.)' , expand = True)
df['title'].unique().sum()

'Mr.Mrs.Miss.Master.Ms.Col.Rev.Dr.Dona.'

In [85]:
df['title'] = df['title'].replace(['Capt.','Col.','Countess.','Don.','Dr.','Jonkheer.','Lady.','Major.','Rev.','Sir.',],'Rare')
df['title'] = df['title'].replace('mme','Mrs.')
df['title'] = df['title'].replace('Mlle.','Miss.')
df['title'] = df['title'].replace('Ms.','Miss.')
df['title'] = df['title'].map({'Master.':1,'Miss':2,'Mr.':3,'Mrs.':4,'Rare':5})
df['title'] = df['title'].fillna(0)

In [86]:

df['Fare'] = df['Fare'].fillna(12.34)

In [87]:
df.Fare.isnull().sum()

0

In [88]:
df.drop('Name',axis=1,inplace=True)
df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,title
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,3.0
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,4.0
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,3.0
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,3.0
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,4.0


In [89]:
df.drop('Ticket',axis=1,inplace= True)

In [90]:
df.drop('Cabin',axis=1,inplace= True)

In [91]:
df['Sex']=df['Sex'].map({'male':1,'female':0})

In [92]:
guess_ages = np.zeros((2,3))
guess_ages
for i in range(0,2):
    for j in range(0,3):
        gussed_age = df[(df['Sex'] == i) & (df['Pclass'] == 1+j)]['Age'].dropna()
        guessed_age = gussed_age.median()
        guess_ages[i,j] = int(guessed_age/0.5 + 0.5)*0.5

In [93]:
for i in range(0, 2):
        for j in range(0, 3):
            df.loc[ (df.Age.isnull()) & (df.Sex == i) & (df.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

In [94]:
print(df[df['Pclass']==1 ]['Fare'].min())
print(df[df['Pclass']==1 ]['Fare'].mean())
print(df[df['Pclass']==1 ]['Fare'].std())
print(df[df['Pclass']==1 ]['Fare'].max())

0.0
94.28029719626169
84.43585836291676
512.3292


In [95]:
df.loc[(df['Age']<=16.336), 'Age'] = 0
df.loc[(df['Age']>16.336)  & (df['Age']<=32.252), 'Age'] = 1
df.loc[(df['Age']>32.252) & (df['Age']<=48.168),'Age'] = 2
df.loc[(df['Age']>48.168) & (df['Age']<=64.084),  'Age'] = 3
df.loc[(df['Age']>64.084) & (df['Age']<=80.0),'Age'] = 4

In [96]:
df.drop('PassengerId',axis=1,inplace = True)

In [97]:
df.loc[(df['Fare']<60) & (df['Pclass']==1), 'Fare'] = df['Fare']+100

In [98]:
df['Familysize'] = df['SibSp'] + df['Parch'] + 1

In [99]:
df['Isalone'] = 0
df.loc[(df['Familysize'] == 1),'Isalone'] = 1

In [100]:
df.drop(['Familysize','SibSp','Parch'],axis=1,inplace = True)

In [101]:
df['fareband']=pd.cut(df['Fare'],4)

In [102]:
# df.loc[(df['Fare']<=128.082),'Fare'] = 0
# df.loc[(df['Fare']>128.082) & (df['Fare']<=256.165),'Fare'] = 1
# df.loc[(df['Fare']>256.165) & (df['Fare']<= 384.247),'Fare'] = 1
# df.loc[(df['Fare']> 384.247) ,'Fare'] = 2

In [103]:
df.loc[(df['Pclass']==2),'Fare'] = 30
df.loc[(df['Pclass']==3),'Fare'] =8
df.loc[(df['Pclass']==1),'Fare'] = 870

In [104]:
df.drop('fareband',inplace= True,axis=1)

In [105]:
freq_port = df['Embarked'].dropna().mode()[0]
df['Embarked'] = df['Embarked'].fillna(freq_port, inplace=False)

In [106]:
df['Embarked'] = df.Embarked.map({'C':2,'S':0,'Q':1}).astype(int)

In [107]:
df.isnull().sum()

Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
title       0
Isalone     0
dtype: int64

In [108]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [109]:
df.head()

,Pclass,Sex,Age,Fare,Embarked,title,Isalone
0,3,1,2.0,8.0,1,3.0,1
1,3,0,2.0,8.0,0,4.0,0
2,2,1,3.0,30.0,1,3.0,1
3,3,1,1.0,8.0,0,3.0,1
4,3,0,1.0,8.0,0,4.0,0


In [110]:
df.to_csv('newone.csv',index=False)